<a href="https://colab.research.google.com/github/alfa7g7/Analisis-Cuantitativo-1er-semestre/blob/main/Examen_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

FACULTAD DE INGENIERÍA, DISEÑO Y CIENCIAS APLICADAS MAESTRÍA EN CIENCIA DE DATOS TIC 60153 – Análisis Cuantitativo Examen 2 (50 %): Modelos de Regularización